In [143]:
import sqlite3
import pyodbc
import pandas as pd

In [144]:
# Conectamos con la base de datos:

connection = sqlite3.connect('data/sql-murder-mystery.db', isolation_level=None)
connection

In [145]:
# Obtenemos un cursor que utilizaremos para hacer las queries
cursor = connection.cursor()
cursor

In [146]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    cursor.execute(query)

    # Almacena los datos de la query 
    ans = cursor.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in cursor.description]

    return pd.DataFrame(ans,columns=names)

In [147]:
    # Obtener información sobre las tablas
try:
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
        tablas = cursor.fetchall()

        # Recorrer las tablas y obtener información sobre cada una
        for tabla in tablas:
            nombre_tabla = tabla[0]
            print("Tabla:", nombre_tabla)

            # Obtener información sobre las columnas de la tabla
            try:
                cursor.execute(f"PRAGMA table_info('{nombre_tabla}')")
                columnas = cursor.fetchall()

                print("Columnas:")
                for columna in columnas:
                    nombre_columna = columna[1]
                    tipo_dato = columna[2]
                    print(f"- {nombre_columna}: {tipo_dato}")

            except sqlite3.Error as error:
                print("Error al obtener información de columnas:", error)

            print()  # Agregar una línea en blanco entre las tablas

except sqlite3.Error as error:
        print("Error al obtener información de tablas:", error)




Tabla: crime_scene_report
Columnas:
- date: integer
- type: text
- description: text
- city: text

Tabla: drivers_license
Columnas:
- id: integer
- age: integer
- height: integer
- eye_color: text
- hair_color: text
- gender: text
- plate_number: text
- car_make: text
- car_model: text

Tabla: person
Columnas:
- id: integer
- name: text
- license_id: integer
- address_number: integer
- address_street_name: text
- ssn: integer

Tabla: facebook_event_checkin
Columnas:
- person_id: integer
- event_id: integer
- event_name: text
- date: integer

Tabla: interview
Columnas:
- person_id: integer
- transcript: text

Tabla: get_fit_now_member
Columnas:
- id: text
- person_id: integer
- name: text
- membership_start_date: integer
- membership_status: text

Tabla: get_fit_now_check_in
Columnas:
- membership_id: text
- check_in_date: integer
- check_in_time: integer
- check_out_time: integer

Tabla: income
Columnas:
- ssn: integer
- annual_income: integer

Tabla: solution
Columnas:
- user: integer

In [148]:
query = "SELECT * FROM crime_scene_report"
sql_query(query)

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City
...,...,...,...,...
1223,20180430,bribery,\n,Garden Grove
1224,20180430,fraud,‘Why not?’ said the March Hare.\n,Houma
1225,20180430,assault,\n,Fontana
1226,20180501,assault,be NO mistake about it: it was neither more no...,Trenton


In [149]:
query = "SELECT * FROM crime_scene_report WHERE city = 'SQL City' AND date = '20180115' AND type = 'murder'"
crime_scene_report = sql_query(query)
crime_scene_report['description'][0]


'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".'

In [150]:
# The first witness lives at the last house on "Northwestern Dr". 
#The second witness, named Annabel, lives somewhere on "Franklin Ave".

In [151]:
query = "SELECT * FROM person WHERE address_street_name = 'Franklin Ave' AND name LIKE '%Annabel%'"
sql_query(query)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [152]:
# 0	16371	Annabel Miller	490173	103	Franklin Ave	318771143

In [153]:
query = '''
SELECT * 
FROM person
WHERE address_street_name = 'Northwestern Dr'
 ORDER BY address_number DESC LIMIT 1
'''
sql_query(query)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [154]:
# 0	14887	Morty Schapiro	118009	4919	Northwestern Dr	111564949

In [155]:
query = '''
SELECT *
FROM interview
WHERE person_id = 14887
OR person_id = 16371'''

interviews = sql_query(query)
print(interviews['transcript'][0])
print(interviews['transcript'][1])

I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".
I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.


In [156]:
interviews

,person_id,transcript
0,14887,I heard a gunshot and then saw a man run out. ...
1,16371,"I saw the murder happen, and I recognized the ..."


In [157]:
# MORTY:  I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. 
#The membership number on the bag started with "48Z". Only gold members have those bags. 
#The man got into a car with a plate that included "H42W".

# ANNABEL:  I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.

In [158]:
query = '''
SELECT *
FROM drivers_license
WHERE plate_number LIKE '%H42W%'
AND gender = 'male'
'''

sql_query(query)


,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [159]:
# 0	423327	30	70	brown	brown	male	0H42W2	Chevrolet	Spark LS
# 1	664760	21	71	black	black	male	4H42WR	Nissan	Altima

In [160]:
query = '''
SELECT *
FROM get_fit_now_member
WHERE membership_status = 'gold'
AND id LIKE '48Z%'
'''

sql_query(query)

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


In [161]:
query = '''
SELECT *
FROM get_fit_now_check_in
WHERE membership_id LIKE '48Z%'
AND check_in_date = 20180109
'''

sql_query(query)

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


In [162]:
query = '''
SELECT *
FROM facebook_event_checkin
WHERE person_id = 28819
OR person_id = 67318
'''

sql_query(query)

,person_id,event_id,event_name,date
0,67318,4719,The Funky Grooves Tour,20180115
1,67318,1143,SQL Symphony Concert,20171206


In [163]:
query = '''
SELECT *
FROM person
WHERE id = 28819
OR id = 67318
'''

sql_query(query)

,id,name,license_id,address_number,address_street_name,ssn
0,28819,Joe Germuska,173289,111,Fisk Rd,138909730
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279


In [164]:
query = '''
SELECT *
FROM income
WHERE ssn = 871539279
'''

sql_query(query)

,ssn,annual_income
0,871539279,10500


In [165]:
query = '''
SELECT *
FROM income
WHERE ssn = 138909730'''

sql_query(query)

,ssn,annual_income


In [177]:
query = '''
INSERT INTO solution (user, value)
VALUES ('1', 'Jeremy Bowers')
'''
cursor.execute(query)
cursor.fetchall()

assasin = sql_query("SELECT * FROM solution")
assasin['value'][0]

"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."

In [167]:
query = '''
SELECT *
FROM interview
WHERE person_id = 67318'''

assasin_interviews = sql_query(query)
assasin_interviews['transcript'][0]

'I was hired by a woman with a lot of money. I don\'t know her name but I know she\'s around 5\'5" (65") or 5\'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n'

In [170]:
query = '''
SELECT *
FROM drivers_license
WHERE hair_color = 'red'
AND car_model = 'Model S'
AND gender = 'female'
'''

sql_query(query)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


In [173]:
query = '''
SELECT *
FROM person
WHERE license_id = 202298
OR license_id = 291182
OR license_id = 918773'''

sql_query(query)

,id,name,license_id,address_number,address_street_name,ssn
0,78881,Red Korb,918773,107,Camerata Dr,961388910
1,90700,Regina George,291182,332,Maple Ave,337169072
2,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [175]:
query = '''
SELECT *
FROM income
WHERE ssn = 961388910
OR ssn = 337169072
OR ssn = 987756388'''

sql_query(query)

,ssn,annual_income
0,961388910,278000
1,987756388,310000


In [176]:
query = '''
SELECT *
FROM facebook_event_checkin
WHERE person_id = 78881
OR person_id = 90700
OR person_id = 99716
'''

sql_query(query)

,person_id,event_id,event_name,date
0,99716,1143,SQL Symphony Concert,20171206
1,99716,1143,SQL Symphony Concert,20171212
2,99716,1143,SQL Symphony Concert,20171229


In [178]:
query = '''
INSERT INTO solution (user, value)
VALUES ('1', 'Miranda Priestly')
'''
cursor.execute(query)
cursor.fetchall()

intelect_assasin = sql_query("SELECT * FROM solution")
intelect_assasin['value'][0]

'Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!'